## ban đầu tôi có một file là text_to_sql và có vài và bước đầu chạy ra kết quả để cho ra truy vấn rồi nhưng mà bị mất, giờ tôi làm file mới


In [4]:
import psycopg2
conn = psycopg2.connect(
    dbname='airflow',
    user='airflow',
    password='airflow',
    host='localhost',
    port='5432'
)
cur = conn.cursor()
print("Connected to the database")

Connected to the database


In [40]:
from sentence_transformers import SentenceTransformer

model_embedding  = SentenceTransformer("multi-qa-mpnet-base-dot-v1", device="cuda")

In [ ]:
import google.generativeai as genai
import os

genai.configure(api_key="gen_api")


In [ ]:
def find_sql(user_text):
    try:
        query_embedding = model_embedding.encode(user_text).tolist()
        cur.execute("""
            SELECT content
            FROM embeddings
            ORDER BY vector <=> %s :: vector
            LIMIT 1;
        """, (query_embedding,))
        results = cur.fetchall()
        if results:
            return results[0][0]  # Trả về content
        else:
            return None
    except Exception as e:
        print("Error:", e)
        conn.rollback()
        return None

In [51]:
text = "sale_fact"
query_embedding = model_embedding.encode(text).tolist()
cur.execute("""
            SELECT content
            FROM embeddings
            ORDER BY vector <=> %s :: vector
            LIMIT 1;
        """, (query_embedding,))
results = cur.fetchall()
print("Results:", results)

Results: [('ngay_goi (timestamp without time zone), sđt (double precision), gio (timestamp without time zone), ngay_hen_goi_lai (timestamp without time zone), close_date (timestamp without time zone), so_lan_tuong_tac (double precision), so_luong_bo_sach (double precision), so_tien_giam_gia (double precision), tong_tien (bigint), trang_thai (text), level (text), tinh/tp (text), sales (text), khach_hang (text), channel (text), chien_dich (text), content (text), marketer_2 (text), type_of_lead (text), sales_admin_xac_nhan_type_of_lead (text), unnamed:_0 (text), ',)]


In [47]:
def generate_sql(user_text):
    sql_context = find_sql(user_text)
    if not sql_context:
        print("Không tìm thấy truy vấn phù hợp.")
        return ""
    model = genai.GenerativeModel("gemini-2.0-flash")
    prompt = str(user_text) + "\n" + str(sql_context)
    response = model.generate_content(prompt)
    sql_query = response.text.strip()
    return sql_query

In [50]:
while True:
    user_input = input("Enter your question: ")
    if user_input.lower() == "exit":
        break
    sql_query = generate_sql(user_input)
    print("Generated SQL Query:", sql_query)

Generated SQL Query: OK. I understand the structure of the `sale_fact` table. Here's a breakdown of the columns and their data types:

*   **ngay\_goi (timestamp without time zone):** Date and time the call was made.
*   **sđt (double precision):** Phone number (likely a numeric representation).  Using `double precision` for a phone number might lead to precision issues; consider using `varchar` or `bigint` if the format is strictly numeric.
*   **gio (timestamp without time zone):** Time of day the call was made.
*   **ngay\_hen\_goi\_lai (timestamp without time zone):** Date and time for a scheduled callback.
*   **close\_date (timestamp without time zone):** Date and time the sale was closed.
*   **so\_lan\_tuong\_tac (double precision):** Number of interactions.
*   **so\_luong\_bo\_sach (double precision):** Number of books sold.
*   **so\_tien\_giam\_gia (double precision):** Discount amount.
*   **tong\_tien (bigint):** Total price.
*   **trang\_thai (text):** Status of the sale